<a href="https://colab.research.google.com/github/SanePai/Neural_Networks-Assignment-2/blob/main/Question_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd "/content/drive/MyDrive/NNFL/Assignment 2"
!ls

/content/drive/.shortcut-targets-by-id/104/Assignment 2
assignment2.pdf  data55.xlsx  input_a2.mat  label.mat
class_label.mat  data5.mat    input.mat


In [ ]:
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
import random
from sklearn.metrics import confusion_matrix as cm

In [ ]:
f = loadmat('data5.mat')
D = f['x']
np.random.shuffle(D)

In [ ]:
def init_data(data):
    X = np.array(data[:2148, :-1], dtype = float)
    y = np.array(data[:2148, -1], dtype = int)
    X = (X - X.mean(axis = 0))/X.std(axis = 0)
    return X, y

def gaussian(x,center,sigma,beta):
    #Gaussian Kernel
    return np.exp(-beta * (np.linalg.norm(x - center)) ** 2)

def multi_quadric(x, center, sigma, beta):
    #Multi Quadratic Kernel
    return ((np.linalg.norm(x - center)) ** 2 + sigma ** 2) ** 0.5

def linear(x, center, sigma, beta):
    #Linear Kernel
    return np.linalg.norm(x - center)

X_tot, y_tot = init_data(D)

train_X = X_tot[ : 1600]
train_y = y_tot[ : 1600]
test_X = X_tot[1600 : 2148]
test_y = y_tot[1600 : 2148]

def fit_rbf(train_X, train_y, test_X, test_y):
    km = KMeans(n_clusters=550)

    y_km = km.fit_predict(train_X)
    centers = km.cluster_centers_
    labels = km.predict(train_X)

    sigma = np.zeros((len(centers), 1))
    beta = np.zeros((len(centers), 1))
    cluster_size = np.zeros((len(centers), 1))

    for i in range(len(train_X)):
        sigma[labels[i]] += np.linalg.norm(train_X[i] - centers[labels[i]])
        cluster_size[labels[i]] += 1

    sigma /= cluster_size
    beta = 1 / 2 * (sigma * sigma + 1e-6)

    H = np.zeros((len(train_X), len(centers)))

    for i in range(len(train_X)):
        for j in range(len(centers)):
            H[i, j] = linear(train_X[i], centers[j], sigma[j], beta[j])

    W = np.dot(np.linalg.pinv(H), train_y)

    #Test run
    H_test = np.zeros([len(test_X), len(centers)])
    for i in range(len(test_X)):
        for j in range(len(centers)):
            H_test[i, j] = linear(test_X[i], centers[j], sigma[j], beta[j])

    y_pred = np.dot(H_test, W)
    for i in range(len(y_pred)):
        y_pred[i] = 1 if y_pred[i]>=0.5 else 0
        
    accuracy = 0    
    for i in range(len(y_pred)):
        if y_pred[i] == test_y[i]:
            accuracy +=1
    accuracy /= len(y_pred)
    print(accuracy)
    return y_pred, accuracy

In [ ]:
y_pred, _ = fit_rbf(train_X, train_y, test_X, test_y)
for i in range(len(y_pred)):
    y_pred[i] = 1 if y_pred[i] > 0.5 else 0

confmat = cm(test_y, y_pred)

Accuracy = (confmat[0][0]+confmat[1][1])/len(y_pred)
Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
print("Confusion Matrix:")
print(confmat)
print("\n")
print(f"Accuracy: {Accuracy}\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}\n")

avg_acc = 0
# K - fold cross validation

for k in range(6):
    X = X_tot[0 : 1790]
    y = y_tot[0 : 1790]
    X_val = X_tot[1790 :]
    y_val = y_tot[1790 :]
    print(f'Fold {k+1}:')
    print("Accuracy: ", end="")
    _, acc = fit_rbf(X, y, X_val, y_val)
    print("===================")
    avg_acc += acc
    X_tot[0 : 358] = X_val
    X_tot[358 : ] = X
    y_tot[0 : 358] = y_val
    y_tot[358 : ] = y

avg_acc /= 6
print(f'\nAverage Accuracy: {avg_acc}')

0.9470802919708029
Confusion Matrix:
[[261  18]
 [ 11 258]]


Accuracy: 0.9470802919708029
Sensitivity: 0.9591078066914498
Specificity: 0.9354838709677419

Fold 1:
Accuracy: 0.9413407821229051
Fold 2:
Accuracy: 0.952513966480447
Fold 3:
Accuracy: 0.9636871508379888
Fold 4:
Accuracy: 0.9553072625698324
Fold 5:
Accuracy: 0.946927374301676
Fold 6:
Accuracy: 0.9357541899441341

Average Accuracy: 0.9492551210428305
